# RAG from scratch

- Adapted from https://github.com/opendatahub-io/llama-stack-demos/
- Requires ollama template 

# Variables

The variables are configured related to the distribution we are connected

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
base_url=os.getenv("BASE_URL") #local "http://localhost:8321"
chunk_size_in_tokens=os.getenv("CHUNK_SIZE_TOKEN") #512
collection_name=os.getenv('COLLECTION_NAME')  ##all-MiniLM-L6-v2"
embedding_model_name=os.getenv('EMBEDDING_MODEL_NAME')  
embedding_size=int(os.getenv('EMBEDDING_SIZE'))   
model_name=os.getenv("MODEL_NAME") 

### Define client and test connection to server

In [65]:
from llama_stack_client import LlamaStackClient
client = LlamaStackClient(
    base_url=base_url,
)

In [66]:
client.models.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/models "HTTP/1.1 200 OK"


[Model(identifier='all-MiniLM-L6-v2', metadata={'embedding_dimension': 384.0}, api_model_type='embedding', provider_id='ollama', type='model', provider_resource_id='all-minilm:latest', model_type='embedding'),
 Model(identifier='llama3.2:3b-instruct-fp16', metadata={}, api_model_type='llm', provider_id='ollama', type='model', provider_resource_id='llama3.2:3b-instruct-fp16', model_type='llm')]

### Ingest data to vector db 

In [68]:
from llama_stack_client import RAGDocument

#urls = ["chat.rst", "llama3.rst", "memory_optimizations.rst", "lora_finetune.rst"]
urls = ["chat.rst", "llama3.rst"]
documents = [
    RAGDocument(
        document_id=f"num-{i}",
        content=f"https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/{url}",
        mime_type="text/plain",
        metadata={},
    )
    for i, url in enumerate(urls)
]

In [50]:
vector_db_id = collection_name
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model=embedding_model_name,
    embedding_dimension=embedding_size,
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/vector-dbs "HTTP/1.1 200 OK"


VectorDBRegisterResponse(embedding_dimension=384, embedding_model='all-MiniLM-L6-v2', identifier='demoInnovate-vectordb', provider_id='faiss', type='vector_db', provider_resource_id='demoInnovate-vectordb', owner={'principal': '', 'attributes': {}})

Ingesting documents into a vector database:

In [52]:
import rich
rich.print(documents)

[
    {
        'document_id': 'num-0',
        'content': 'https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/chat.rst',
        'mime_type': 'text/plain',
        'metadata': {}
    },
    {
        'document_id': 'num-1',
        'content': 'https://raw.githubusercontent.com/pytorch/torchtune/main/docs/source/tutorials/llama3.rst',
        'mime_type': 'text/plain',
        'metadata': {}
    }
]

In [53]:
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=chunk_size_in_tokens,
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/rag-tool/insert "HTTP/1.1 200 OK"


## Testing vector db 

In [63]:
prompt = "What are the top 5 topics that were explained? Only list succinct bullet points."

### Using tool_runtime api that can query in multiple collection

In [55]:
# higher level tool provides packaged results, can span multiple dbs
tool_response = client.tool_runtime.rag_tool.query(
    content=prompt, vector_db_ids=[vector_db_id]
)
rich.print(tool_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/rag-tool/query "HTTP/1.1 200 OK"


QueryResult(
    metadata={'document_ids': ['num-0', 'num-1', 'num-0', 'num-0', 'num-0']},
    content=[
        TextContentItem(
            text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n',
            type='text'
        ),
        TextContentItem(
            text='Result 1\nContent:  Instruct.\n\n.. _prompt_template_vs_special_tokens:\n\nTokenizing prompt 
templates & special tokens\n--------------------------------------------\n\nLet\'s say I have a sample of a single 
user-assistant turn accompanied with a system\nprompt:\n\n.. code-block:: python\n\n    sample = [\n        {\n    
"role": "system",\n            "content": "You are a helpful, respectful, and honest assistant.",\n        },\n    
{\n            "role": "user",\n            "content": "Who are the most influential hip-hop artists of all 
time?",\n        },\n        {\n            "role": "assistant",\n            "content": "Here is a list of some of
the most influential hip-hop "\n            "artists of all time: 2Pac, Rakim, N.W.A., Run-D.M.C., and Nas.",\n    
},\n    ]\n\nNow, let\'s format this with the :class:`~torchtune.models.llama2.Llama2ChatTemplate` class and\nsee 
how it gets tokenized. The Llama2ChatTemplate is an example of a **prompt template**,\nwhich simply structures a 
prompt with flavor text to indicate a certain task.\n\n.. code-block:: python\n\n    from torchtune.data import 
Llama2ChatTemplate, Message\n\n    messages = [Message.from_dict(msg) for msg in sample]\n    formatted_messages = 
Llama2ChatTemplate.format(messages)\n    print(formatted_messages)\n    # [\n    #     Message(\n    #         
role=\'user\',\n    #         content=\'[INST] <<SYS>>\\nYou are a helpful, respectful, and honest 
assistant.\\n<</SYS>>\\n\\nWho are the most influential hip-hop artists of all time? [/INST] \',\n    #         
...,\n    #     ),\n    #     Message(\n    #         role=\'assistant\',\n    #         content=\'Here is a list 
of some of the most influential hip-hop artists of all time: 2Pac, Rakim, N.W.A., Run-D.M.C., and Nas.\',\n    #   
...,\n    #     ),\n    # ]\n\nThere are also special tokens used by Llama2, which are not in the prompt 
template.\nIf you look at our :class:`~torchtune.models.llama2.Llama2ChatTemplate` class, you\'ll notice that\nwe 
don\'t include the :code:`<s>` and :code:`</s>` tokens. These are the beginning-of-sequence\n(BOS) and 
end-of-sequence (EOS) tokens that are represented differently\nMetadata: {\'document_id\': \'num-0\'}\n',
            type='text'
        ),
        TextContentItem(
            text='Result 2\nContent:  VRAM, and in fact the QLoRA recipe should have peak allocated memory\nbelow 
10 GB. You can also experiment with different configurations of LoRA and QLoRA, or even run a full fine-tune.\nTry 
it out!\n\n|\n\nEvaluating fine-tuned Llama3-8B models with EleutherAI\'s Eval 
Harness\n---------------------------------------------------------------------\n\nNow that we\'ve fine-tuned our 
model, what\'s next? Let\'s take our LoRA-finetuned model from the\npreceding section and look at a couple 
different ways we can evaluate its performance on the tasks we care about.\n\nFirst, torchtune provides an 
integration with\n`EleutherAI\'s evaluation harness <https://github.com/EleutherAI/lm-evaluation-harness>`_\nfor 
model evaluation on common benchmark tasks.\n\n.. note::\n    Make sure you\'ve first installed the evaluation 
harness via :code:`pip install "lm_eval==0.4.*"`.\n\nFor this tutorial we\'ll use the `truthfulqa_mc2 
<https://github.com/sylinrl/TruthfulQA>`_ task from the harness.\nThis task measures a model\'s propensity to be 
truthful when answering questions and\nmeasures the model\'s zero-shot accuracy on a question followed by one or 
more true\nresponses and one or more false responses. First, let\'s copy the config so we can point the YAML\nfile 
to our fine-tuned checkpoint files.\n\n.. code-block:: bash\n\n    tune cp eleuther_evalu

### Query the vector io collection directly

In [56]:
# we can also query the vector db directly
db_response = client.vector_io.query(
    vector_db_id=vector_db_id,
    query=prompt,
    
)
rich.print(db_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/vector-io/query "HTTP/1.1 200 OK"


QueryChunksResponse(
    chunks=[
        Chunk(
            content=' Instruct.\n\n.. _prompt_template_vs_special_tokens:\n\nTokenizing prompt templates & special 
tokens\n--------------------------------------------\n\nLet\'s say I have a sample of a single user-assistant turn 
accompanied with a system\nprompt:\n\n.. code-block:: python\n\n    sample = [\n        {\n            "role": 
"system",\n            "content": "You are a helpful, respectful, and honest assistant.",\n        },\n        {\n 
"role": "user",\n            "content": "Who are the most influential hip-hop artists of all time?",\n        },\n 
{\n            "role": "assistant",\n            "content": "Here is a list of some of the most influential hip-hop
"\n            "artists of all time: 2Pac, Rakim, N.W.A., Run-D.M.C., and Nas.",\n        },\n    ]\n\nNow, let\'s 
format this with the :class:`~torchtune.models.llama2.Llama2ChatTemplate` class and\nsee how it gets tokenized. The
Llama2ChatTemplate is an example of a **prompt template**,\nwhich simply structures a prompt with flavor text to 
indicate a certain task.\n\n.. code-block:: python\n\n    from torchtune.data import Llama2ChatTemplate, 
Message\n\n    messages = [Message.from_dict(msg) for msg in sample]\n    formatted_messages = 
Llama2ChatTemplate.format(messages)\n    print(formatted_messages)\n    # [\n    #     Message(\n    #         
role=\'user\',\n    #         content=\'[INST] <<SYS>>\\nYou are a helpful, respectful, and honest 
assistant.\\n<</SYS>>\\n\\nWho are the most influential hip-hop artists of all time? [/INST] \',\n    #         
...,\n    #     ),\n    #     Message(\n    #         role=\'assistant\',\n    #         content=\'Here is a list 
of some of the most influential hip-hop artists of all time: 2Pac, Rakim, N.W.A., Run-D.M.C., and Nas.\',\n    #   
...,\n    #     ),\n    # ]\n\nThere are also special tokens used by Llama2, which are not in the prompt 
template.\nIf you look at our :class:`~torchtune.models.llama2.Llama2ChatTemplate` class, you\'ll notice that\nwe 
don\'t include the :code:`<s>` and :code:`</s>` tokens. These are the beginning-of-sequence\n(BOS) and 
end-of-sequence (EOS) tokens that are represented differently',
            metadata={'document_id': 'num-0', 'token_count': 512.0, 'metadata_token_count': 1.0},
            embedding=[
                0.027858738,
                0.053094324,
                -0.030784557,
                0.016754113,
                -0.013624205,
                0.100132704,
                0.027882783,
                0.0072661764,
                -0.03545826,
                -0.013629367,
                -0.02712671,
                -0.013454738,
                -0.027180888,
                0.0057096086,
                0.07389063,
                0.11050068,
                -0.012866331,
                0.043541733,
                -0.052139707,
                -0.07226332,
                0.04546949,
                -0.0057796584,
                0.0902325,
                0.0020647778,
                -0.010973703,
                0.053373404,
                0.0026874188,
                0.059748426,
                0.03890687,
                -0.0223437,
                0.001896622,
                0.017173035,
                -0.013574308,
                0.06605402,
                -0.04088732,
                0.0029724005,
                0.039622743,
                0.062411536,
                0.043182693,
                0.039786395,
                0.096769854,
                0.05104551,
                -0.034624334,
                -0.07569449,
                0.009018039,
                -0.13605429,
                -0.04451267,
                -0.021502225,
                -0.07213174,
                0.039564762,
                -0.01211748,
                -0.09419846,
                -0.010627737,
                0.027582401,
                0.014288939,
                0.0069212355,
         

## Creating context joining all the search

In [57]:
# prompt_context = tool_response.content
prompt_context = "\n".join([c.content for c in db_response.chunks])

In [58]:
messages = [{"role": "system", "content": "You are a helpful assistant."}]
extended_prompt = f"""
Please answer the given query using the context below.

QUERY:
{prompt}

CONTEXT:
{prompt_context}
"""
messages.append({"role": "user", "content": extended_prompt})
rich.print(messages)

[
    {'role': 'system', 'content': 'You are a helpful assistant.'},
    {
        'role': 'user',
        'content': '\nPlease answer the given query using the context below.\n\nQUERY:\nWhat are the top 5 topics 
that were explained? Only list succinct bullet points.\n\nCONTEXT:\n Instruct.\n\n.. 
_prompt_template_vs_special_tokens:\n\nTokenizing prompt templates & special 
tokens\n--------------------------------------------\n\nLet\'s say I have a sample of a single user-assistant turn 
accompanied with a system\nprompt:\n\n.. code-block:: python\n\n    sample = [\n        {\n            "role": 
"system",\n            "content": "You are a helpful, respectful, and honest assistant.",\n        },\n        {\n 
"role": "user",\n            "content": "Who are the most influential hip-hop artists of all time?",\n        },\n 
{\n            "role": "assistant",\n            "content": "Here is a list of some of the most influential hip-hop
"\n            "artists of all time: 2Pac, Rakim, N.W.A., Run-D.M.C., and Nas.",\n        },\n    ]\n\nNow, let\'s 
format this with the :class:`~torchtune.models.llama2.Llama2ChatTemplate` class and\nsee how it gets tokenized. The
Llama2ChatTemplate is an example of a **prompt template**,\nwhich simply structures a prompt with flavor text to 
indicate a certain task.\n\n.. code-block:: python\n\n    from torchtune.data import Llama2ChatTemplate, 
Message\n\n    messages = [Message.from_dict(msg) for msg in sample]\n    formatted_messages = 
Llama2ChatTemplate.format(messages)\n    print(formatted_messages)\n    # [\n    #     Message(\n    #         
role=\'user\',\n    #         content=\'[INST] <<SYS>>\\nYou are a helpful, respectful, and honest 
assistant.\\n<</SYS>>\\n\\nWho are the most influential hip-hop artists of all time? [/INST] \',\n    #         
...,\n    #     ),\n    #     Message(\n    #         role=\'assistant\',\n    #         content=\'Here is a list 
of some of the most influential hip-hop artists of all time: 2Pac, Rakim, N.W.A., Run-D.M.C., and Nas.\',\n    #   
...,\n    #     ),\n    # ]\n\nThere are also special tokens used by Llama2, which are not in the prompt 
template.\nIf you look at our :class:`~torchtune.models.llama2.Llama2ChatTemplate` class, you\'ll notice that\nwe 
don\'t include the :code:`<s>` and :code:`</s>` tokens. These are the beginning-of-sequence\n(BOS) and 
end-of-sequence (EOS) tokens that are represented differently\n VRAM, and in fact the QLoRA recipe should have peak
allocated memory\nbelow 10 GB. You can also experiment with different configurations of LoRA and QLoRA, or even run
a full fine-tune.\nTry it out!\n\n|\n\nEvaluating fine-tuned Llama3-8B models with EleutherAI\'s Eval 
Harness\n---------------------------------------------------------------------\n\nNow that we\'ve fine-tuned our 
model, what\'s next? Let\'s take our LoRA-finetuned model from the\npreceding section and look at a couple 
different ways we can evaluate its performance on the tasks we care about.\n\nFirst, torchtune provides an 
integration with\n`EleutherAI\'s evaluation harness <https://github.com/EleutherAI/lm-evaluation-harness>`_\nfor 
model evaluation on common benchmark tasks.\n\n.. note::\n    Make sure you\'ve first installed the evaluation 
harness via :code:`pip install "lm_eval==0.4.*"`.\n\nFor this tutorial we\'ll use the `truthfulqa_mc2 
<https://github.com/sylinrl/TruthfulQA>`_ task from the harness.\nThis task measures a model\'s propensity to be 
truthful when answering questions and\nmeasures the model\'s zero-shot accuracy on a question followed by one or 
more true\nresponses and one or more false responses. First, let\'s copy the config so we can point the YAML\nfile 
to our fine-tuned checkpoint files.\n\n.. code-block:: bash\n\n    tune cp eleuther_evaluation 
./custom_eval_config.yaml\n\nNext, we modify ``custom_eval_config.yaml`` to include the fine-tuned 
checkpoints.\n\n.. code-block:: yaml\n\n    model:\n      _component_: torchtune.mo

In [59]:
response = client.inference.chat_completion(
    messages=messages,
    model_id=model_name,
    timeout=600
)
rich.print(response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/inference/chat-completion "HTTP/1.1 200 OK"


ChatCompletionResponse(
    completion_message=CompletionMessage(
        content='* Tokenizing prompt templates & special tokens\n* Llama3 Instruct \n* Prompt template vs special 
tokens',
        role='assistant',
        stop_reason='end_of_turn',
        tool_calls=[]
    ),
    logprobs=None,
    metrics=[
        Metric(metric='prompt_tokens', value=1591.0, unit=None),
        Metric(metric='completion_tokens', value=32.0, unit=None),
        Metric(metric='total_tokens', value=1623.0, unit=None)
    ]
)

### Using Agent 

In [60]:
# Left as an extra exercise for the reader
from llama_stack_client import Agent

rag_agent = Agent(
    client, 
    model=model_name,
    instructions="You are a helpful assistant",
    tools = [
        {
          "name": "builtin::rag/knowledge_search",
          "args" : {
            "vector_db_ids": [vector_db_id],
          }
        }
    ],
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=builtin%3A%3Arag%2Fknowledge_search "HTTP/1.1 200 OK"


In [62]:
import uuid
from llama_stack_client.lib.agents.event_logger import EventLogger
user_prompts = [
    "How I can fine tune Llama 3 with Chat data?"
]
session_id = rag_agent.create_session(f"rag session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")
    response = rag_agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )
    for log in EventLogger().log(response):
        log.print()

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/31a04c8b-e542-43e2-9a40-15618f0dc64c/session "HTTP/1.1 200 OK"


User> How I can fine tune Llama 3 with Chat data?

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/31a04c8b-e542-43e2-9a40-15618f0dc64c/session/b3b1c2d7-4d1f-4119-b136-06e8bbbdbeb5/turn "HTTP/1.1 200 OK"


inference> [knowledge_search(query="fine-tuning Llama 3 with chat data")]
tool_execution> Tool:knowledge_search Args:{'query': 'fine-tuning Llama 3 with chat data'}
tool_execution> Tool:knowledge_search Response:[TextContentItem(text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n', type='text'), TextContentItem(text='Result 1\nContent:  a lightweight structure to prime your fine-tuned model for prompts asking to summarize text.\nThis would wrap around the user message, with the assistant message untouched.\n\n.. code-block:: python\n\n    f"Summarize this dialogue:\\n{dialogue}\\n---\\nSummary:\\n"\n\nYou can fine-tune Llama2 with this template even though the model was originally pre-trained\nwith the :class:`~torchtune.models.llama2.Llama2ChatTemplate`, as long as this is what the model\nsees during inference. The model should be robust enough to adapt to a new template.\n\n\nFine-tuning on a custom chat dataset\n------------------------------------